<a href="https://colab.research.google.com/github/XTRA26/MiXeM/blob/master/Time_Travel_Rephotography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SIGGRAPH Asia 2021] Time-Travel Rephotography

**Author**: [Xuan Luo](https://roxanneluo.github.io/)

[[Project Page](https://time-travel-rephotography.github.io/)]
[[Paper](https://arxiv.org/abs/2012.12261)]
[[Video](https://youtu.be/ceIopN2UZ_s)]
[[GitHub](https://github.com/Time-Travel-Rephotography/Time-Travel-Rephotography.github.io.git)]

If you find this work useful, please consider citing:
```bibtex
@article{Luo-Rephotography-2021,
  author    = {Luo, Xuan and Zhang, Xuaner and Yoo, Paul and Martin-Brualla, Ricardo and Lawrence, Jason and Seitz, Steven M.},
  title     = {Time-Travel Rephotography},
  journal = {ACM Transactions on Graphics (Proceedings of ACM SIGGRAPH Asia 2021)},
  publisher = {ACM New York, NY, USA},
  volume = {40},
  number = {6},
  articleno = {213},
  doi = {https://doi.org/10.1145/3478513.3480485},
  year = {2021},
  month = {12}
}

```

## Clone repository

In [ ]:
!!git clone  --depth 1 --recurse-submodules --shallow-submodules \
  https://github.com/Time-Travel-Rephotography/Time-Travel-Rephotography.github.io.git Time-Travel-Rephotography
%cd Time-Travel-Rephotography

/content/Time-Travel-Rephotography


## Install dependencies

In [ ]:
!pip3 install -r requirements.txt

## Download models

In [ ]:
%%bash

set -exo

mkdir -p checkpoint
gdown https://drive.google.com/uc?id=1YLB-3ZCv6FRAWwCHkTvUou_CAvDcb5Pr -O checkpoint/e4e_ffhq_encode.pt
gdown https://drive.google.com/uc?id=1aSnTVHGNzQ-Eh5rNDOrOc_4SjQbNMq3B -O checkpoint/stylegan2-ffhq-config-f.pt
gdown https://drive.google.com/uc?id=12BHlsSQM0D8KyXprIc7WmJRLWBWdtF56 -O checkpoint/vgg_face_dag.pt
mkdir -p checkpoint/encoder
gdown https://drive.google.com/uc?id=1fXBCyBKNEZfeiI6LEHTEnUg_DjWHJy5r -O checkpoint/encoder/checkpoint_b.pt
gdown https://drive.google.com/uc?id=1YnQEPf7FyfZxAVQg-CpbvdF4otZeTOZh -O checkpoint/encoder/checkpoint_g.pt
gdown https://drive.google.com/uc?id=1kelQK3pUdeHwu8uVE7A4WZH_EjHxGE6v -O checkpoint/encoder/checkpoint_gb.pt

pushd third_party/face_parsing
  mkdir -p res/cp
  gdown https://drive.google.com/uc?id=1vwm4BcAKISQgcJLvTUcvesk73UIYdDMF -O res/cp/79999_iter.pth
popd


allexport      	off
braceexpand    	on
emacs          	off
errexit        	on
errtrace       	off
functrace      	off
hashall        	on
histexpand     	off
history        	off
ignoreeof      	off
interactive-comments	on
keyword        	off
monitor        	off
noclobber      	off
noexec         	off
noglob         	off
nolog          	off
notify         	off
nounset        	off
onecmd         	off
physical       	off
pipefail       	off
posix          	off
privileged     	off
verbose        	off
vi             	off
xtrace         	on
++ mkdir -p checkpoint
++ gdown 'https://drive.google.com/uc?id=1hWc2JLM58_PkwfLG23Q5IH3Ysj2Mo1nr' -O checkpoint/e4e_ffhq_encode.pt
Downloading...
From: https://drive.google.com/uc?id=1hWc2JLM58_PkwfLG23Q5IH3Ysj2Mo1nr
To: /content/Time-Travel-Rephotography/checkpoint/e4e_ffhq_encode.pt
100% 1.20G/1.20G [00:05<00:00, 203MB/s]
++ gdown 'https://drive.google.com/uc?id=1hvAAql9Jo0wlmLBSHRIGrtXHcKQE-Whn' -O checkpoint/stylegan2-ffhq-config-f.pt
Downloading...
F

If the script above fails, which is largely due to exceeding download quota, try run the script below.

```
!./scripts/download_checkpoints.sh
```

## Configure Input

In [ ]:
from pathlib import Path
import os


def upload_files(input_dir=Path()):
  from google.colab import files
  input_dir.mkdir(parents=True, exist_ok=True)
  prev_dir = os.getcwd()
  os.chdir(input_dir)
  uploaded = files.upload()
  os.chdir(prev_dir)
  return [input_dir / k for k in uploaded.keys()]


# @title Path to the input image
input_path = "dataset/Abraham Lincoln_01.png"  # @param {type: 'string'}

# @markdown Do you want to upload and process your own image instead?
upload = False #@param {type:"boolean"}

if upload:
  from tools.data import align_images as align
  # upload image
  upload_dir = Path("dataset/unaligned")
  paths = upload_files(upload_dir)
  # align image
  aligned_dir = Path("dataset/")
  args = align.parse_args([
    str(upload_dir),
    str(aligned_dir),
  ])
  align.main(args)
  input_path = aligned_dir / f"{paths[0].stem}_01.png"

print(f"input_path = '{input_path}'.")


input_path = 'dataset/Abraham Lincoln_01.png'.


In [ ]:
# @title Spectral sensitivity of the negative
# @markdown The `spectral_sensitivity` can be `'b'` (blue-sensitive), `'gb'` (orthochromatic), or `'g'` (panchromatic). 
# @markdown You can roughly estimate the spectral_sensitivity of your photo as follows. Use the blue-sensitive model for photos before 1873, manually select between blue-sensitive and orthochromatic for images from 1873 to 1906 and among all models for photos taken afterwards.
spectral_sensitivity = "b"  # @param ["b", "gb", "g"]


In [ ]:
# @title Estimated blur radius of the input photo
gaussian_radius = 0.75  # @param {type:"number"}

## Run

In [ ]:
%load_ext autoreload
%autoreload

from argparse import Namespace
from projector import (
    ProjectorArguments,
    main,
)

args = ProjectorArguments().parse(
    args=[str(input_path)], 
    namespace=Namespace(
        spectral_sensitivity=spectral_sensitivity,
        encoder_ckpt=f"checkpoint/encoder/checkpoint_{spectral_sensitivity}.pt",
        encoder_name=spectral_sensitivity,
        gaussian=gaussian_radius,
        log_visual_freq=1000,
        log_dir="log/",
        results_dir="results/"
))
main(args)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
------------ Parameters -------------
camera_lr: 0.01
ckpt: checkpoint/stylegan2-ffhq-config-f.pt
coarse_min: 32
color_transfer: 10000000000.0
contextual: 0.1
cx_layers: ['relu3_4', 'relu2_2', 'relu1_2']
e4e_ckpt: checkpoint/e4e_ffhq_encode.pt
e4e_size: 256
encoder_ckpt: checkpoint/encoder/checkpoint_b.pt
encoder_name: b
encoder_size: 256
eye: 0.1
gaussian: 0.75
generator_size: 1024
init_latent: None
input: dataset/Abraham Lincoln_01.png
log_dir: log/
log_freq: 10
log_visual_freq: 1000
lr: 0.1
mix_layer_range: [10, 18]
noise_ramp: 0.75
noise_regularize: 50000.0
noise_strength: 0.0
rand_seed: None
recon_size: 256
results_dir: results/
spectral_sensitivity: b
vgg: 1
vggface: 0.3
wplus_step: [250, 750]
-------------------------------------
b-G0.75-init(10,18)-s256-vgg1-vggface0.3-eye0.1-color1.0e+10-cx0.1(relu3_4,relu2_2,relu1_2)-NR5.0e+04-lr0.1_0.01-c32-wp(250,750)
Loading e4e over the pSp framework f

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


python test.py /content/Time-Travel-Rephotography/results/Abraham Lincoln_01/input_sibling /content/Time-Travel-Rephotography/results/Abraham Lincoln_01/skin_mask/face_parsing


100%|██████████| 2/2 [00:00<00:00, 164.88it/s]


Optimizing 32x32


face:  3.573e+00; eye:  4.444e+00; contextual:  8.362e+00; color_transfer:  1.014e-23; noise:  3.251e-01:   0%|          | 0/250 [00:01<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)
face:  2.333e+00; eye:  3.871e+00; contextual:  9.569e+00; color_transfer:  6.883e-13; noise:  1.350e-03: 100%|██████████| 250/250 [05:15<00:00,  1.26s/it]


Optimizing 64x64


face:  1.748e+00; eye:  3.529e+00; contextual:  7.939e+00; color_transfer:  2.042e-12; noise:  3.290e-05: 100%|██████████| 750/750 [15:43<00:00,  1.26s/it]
